# NLP Vocabulary matching  
[GitHUb repo](https://github.com/runo-hr/NLP-vocabulary-matching)  

This notebook outlines the process of matching similar agricultural commodities from two different sources.

## Importing Libraries

In [ ]:
#! pip install thefuzz
#! pip install python-Levenshtein

In [1]:
import pandas as pd
from thefuzz import fuzz, process

In [ ]:
import sys
# The recursion limit is 10 ** 3. The input data is in the magnitude of 10 ** 4.
# The matching algorithm will be recursive.
sys.setrecursionlimit(10**6)

## Understanding the Data

In [2]:
df1 = pd.read_csv('source_1.csv')
df2 = pd.read_csv('source_2.csv')

print(f'Shape of source_1 : {df1.shape}')
print(f'Shape of source_2 : {df2.shape}')

Shape of source_1 : (13238, 2)
Shape of source_2 : (48943, 2)


In [3]:
df1.head()

,id,name
0,0,"Horses, asses, mules and hinnies; live, pure-b..."
1,1,"Horses; live, pure-bred breeding animals"
2,2,"Horses; live, other than pure-bred breeding an..."
3,4,"Horses, asses, mules and hinnies; live, other ..."
4,5,"Bovine animals; live, pure-bred breeding animals"


In [4]:
df2.head()

,id,name
0,0,leveillula lactucae-serriolae
1,1,podosphaera aphanis
2,2,lathyrus czeczottianus
3,3,crocus biflorus subsp. caricus
4,4,hordeum brevisubulatum


In [5]:
df1 = df1[['id', 'name']].set_index('name')
df1.index.name = None
s1_dict = df1.to_dict()['id']

df2 = df2[['id', 'name']].set_index('name')
df2.index.name = None
s2_dict = df2.to_dict()['id']

print(f"There are {len(s1_dict)} unique commodities in the first source. {len(df1) - len(s1_dict)} were duplicates.")
print(f"There are {len(s2_dict)} unique commodities in the second source. {len(df2) - len(s2_dict)} were duplicates.")

There are 12582 unique commodities in the first source. 656 were duplicates.
There are 48874 unique commodities in the second source. 69 were duplicates.


### Insights
* Source_1 entries are generally longer than source_2.
* There were 12582 unique entries in source_1 and 48874 entries in source_2.

### New data
* df1: Source 1 dataframe, df2: source 2 dataframe.
* s1_dict and s2_dict are name-id dictionaries of commodities from the two sources

The code cells below create input lists for the matching algorithm.  
The jupyter kernel failed before processing all the entries. To rerun the matching algorithm, I removed the already matched examples from the input list.

In [ ]:
df_old_matches = pd.read_csv('matched_names_ids.csv')
df_old_matches.head()

In [ ]:
df1_old = df_old_matches[['id1', 'source_1']].set_index('source_1')
df1_old_dict = df1_old.to_dict()['id1']

df2_old = df_old_matches[['id2', 'source_2']].set_index('source_2')
df2_old_dict = df2_old.to_dict()['id2']


In [ ]:
df1.index.to_list()[:10]

In [ ]:
s1 = []
for entry in df1.index.to_list():
    try:
        df1_old_dict[entry]
    except KeyError:
        s1.append(entry)
#s1

In [ ]:
df2.index.to_list()[:10]

In [ ]:
s2 = []
for entry in df2.index.to_list():
    try:
        df2_old_dict[entry]
    except KeyError:
        s2.append(entry)
#s2

## Matching algorithm  
The algorithm uses the extractOne method from the fuzz class of thefuzz library.
extractOne gets the best match of a string in a list of strings.  
The scorer parameter is passed to tune the results of best match.  
Below, I used the partial_token_sort_ratio as the scorer. I found it to give the best matches for this task.  

Other scorers include:  
* partial_token_sort_ratio (used below)
* partial_token_set_ratio  
* token_sort_ratio
* token_set_ratio 
* partial_ratio
* ratio

More on fuzzy string matching [here](https://youtu.be/1jNNde4k9Ng).

In [ ]:
# lists to create pandas dataframes
matched_ids = []
matched_names_ids = []

def closest_matches(src1:list, src2:list, names_id_src1:dict, names_id_src2:dict):
    '''
        Parameters
        src1:list, src2:list, 
        names_id_src1:dict, names_id_src2:dict
        
        return
        matched_ids:list,  matched_names_ids:list
    '''
    # save unprocessed inputs to csv incase the runtime disconnects
    s1_df = pd.DataFrame(src1)
    s1_df.to_csv('remaininder_s1.csv')

    s2_df = pd.DataFrame(src2)
    s2_df.to_csv('remaininder_s2.csv')
    
    # base case of recursion
    # source 1 has fewer entries than source 2
    if len(src1) == 0:
        return matched_ids, matched_names_ids
    
    for item in src1:
        best_match = process.extractOne(item, src2, scorer=fuzz.partial_token_sort_ratio)
        confidence = best_match[1] # confidence of the match
        
        if confidence == 100: # good match
            src1_item_id = names_id_src1[item] # get item id

            src2_item = best_match[0] # get source 2 item
            src2_item_id = names_id_src2[src2_item] # source 2 item id

            # save matched ids to csv
            matched_ids.append(dict(source_1=src1_item_id, source_2=src2_item_id))
            df_ids = pd.DataFrame(matched_ids)
            df_ids.to_csv('predicted_ids.csv')

            # save matched ids and names to csv
            matched_names_ids.append(dict(id1=src1_item_id, source_1=item, id2=src2_item_id,source_2=src2_item))
            df_names_ids = pd.DataFrame(matched_names_ids)
            df_names_ids.to_csv('predicted_names_ids.csv')

            # remove found matches from input lists
            src1.remove(item)
            src2.remove(src2_item)

            print(df_names_ids)
            print()
            return closest_matches(src1, src2, names_id_src1, names_id_src2)
        else: # no good match found
            src1.remove(item)
            return closest_matches(src1, src2, names_id_src1, names_id_src2)
    

In [ ]:
len(s1), len(s2), len(s1_dict), len(s2_dict)

In [ ]:
ids_matched, names_ids_matched = closest_matches(s1, s2, s1_dict, s2_dict)

## Further Processing  
Some predicted matches were wrong.  
The cells below go through every found match and determine the token_set_ratio.   
Matches with less than 100 are discarded.  

In [215]:
def get_ratios(i, j):
    print(f"'{i}' vs '{j}'")

    ratio = fuzz.ratio(i, j)
    print(f"Ratio : {ratio}")

    partial_ratio = fuzz.partial_ratio(i, j)
    print(f"Partial Ratio : {partial_ratio}")

    token_sort_ratio = fuzz.token_sort_ratio(i, j)
    print(f"Token sort Ratio : {token_sort_ratio}")

    token_set_ratio = fuzz.token_set_ratio(i, j)
    print(f"Token set Ratio : {token_set_ratio}")

    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(i, j)
    print(f"Partial token sort Ratio : {partial_token_sort_ratio}")

    partial_token_set_ratio = fuzz.partial_token_set_ratio(i, j)
    print(f"Partial token set Ratio : {partial_token_set_ratio}")
    print()


    return partial_ratio, token_set_ratio

In [216]:
#df_matched = pd.read_csv('matched_names_ids.csv')
df_matched = pd.read_csv('predicted_names_ids3.csv')
df_matched.shape

(94, 5)

In [217]:
df_matched

,Unnamed: 0,id1,source_1,id2,source_2
0,0,11198,Dynamometers,8946,dynamometers
1,1,11199,Surface-finish testing instruments,42547,testing
2,2,11200,Machines for taking three-dimensional measurem...,32546,measurement
3,3,11209,Artificial honeycombs,21116,honeycombs
4,4,11219,Hand sieves and hand riddles,11728,sieves
...,...,...,...,...,...
89,89,12129,Appliances known as iron lungs,30164,lungs
90,90,12130,Ozone therapy apparatus and oth.of therapeutic...,37661,ozone
91,91,12141,Reception appar.personal of radium-mes.in scre...,39826,radium
92,92,12183,Electronic microassemblies,36565,assemblies


In [218]:
s1 = df_matched.source_1.to_list()
s2 = df_matched.source_2.to_list()

In [219]:
len(s1_dict), len(s2_dict)

(12582, 48874)

In [220]:
processed_ids = []
processed_names_ids = []
for x,y in zip(s1, s2):
    partial_ratio, token_set_ratio = get_ratios(x,y)
    if (token_set_ratio < 90 and partial_ratio < 80) :
        continue
    else:
        id1 = s1_dict[x]
        id2 = s2_dict[y]
        
        processed_ids.append(dict(source_1=id1, source_2=id2))
        df_processed_ids = pd.DataFrame(processed_ids)
        df_processed_ids.to_csv('processed_ids3.csv')
            
        processed_names_ids.append(dict(id_1=id1, source_1=x, id_2=id2, source_2=y))
        df_processed_names_ids = pd.DataFrame(processed_names_ids)
        df_processed_names_ids.to_csv('processed_names_ids3.csv')

'Dynamometers' vs 'dynamometers'
Ratio : 92
Partial Ratio : 92
Token sort Ratio : 100
Token set Ratio : 100
Partial token sort Ratio : 100
Partial token set Ratio : 100

'Surface-finish testing instruments' vs 'testing'
Ratio : 34
Partial Ratio : 100
Token sort Ratio : 34
Token set Ratio : 100
Partial token sort Ratio : 100
Partial token set Ratio : 100

'Machines for taking three-dimensional measurements' vs 'measurement'
Ratio : 36
Partial Ratio : 100
Token sort Ratio : 36
Token set Ratio : 36
Partial token sort Ratio : 100
Partial token set Ratio : 100

'Artificial honeycombs' vs 'honeycombs'
Ratio : 65
Partial Ratio : 100
Token sort Ratio : 65
Token set Ratio : 100
Partial token sort Ratio : 100
Partial token set Ratio : 100

'Hand sieves and hand riddles' vs 'sieves'
Ratio : 35
Partial Ratio : 100
Token sort Ratio : 35
Token set Ratio : 100
Partial token sort Ratio : 100
Partial token set Ratio : 100

'Autom.electronic controllers f/anti-lock braking syst.' vs 'raking'
Ratio : 20


### Combine the two processed matched csvs

In [221]:
dfp1 = pd.read_csv('runoharris215@gmail.com.csv')
dfp2 = pd.read_csv('processed_ids3.csv')


In [222]:
del(dfp1['Unnamed: 0'])
del(dfp2['Unnamed: 0'])

In [223]:
dfp1.tail()

,source_1,source_2
2223,11991,20106
2224,11995,8397
2225,11999,42414
2226,12014,33961
2227,12041,4663


In [224]:
dfp2.tail()

,source_1,source_2
88,12129,30164
89,12130,37661
90,12141,39826
91,12183,36565
92,12188,7013


In [225]:
frames = [dfp1, dfp2]
result = pd.concat(frames)

In [226]:
result.tail()

,source_1,source_2
88,12129,30164
89,12130,37661
90,12141,39826
91,12183,36565
92,12188,7013


In [227]:
result.reset_index(inplace=True)

In [228]:
result.tail()

,index,source_1,source_2
2316,88,12129,30164
2317,89,12130,37661
2318,90,12141,39826
2319,91,12183,36565
2320,92,12188,7013


In [229]:
result.columns

Index(['index', 'source_1', 'source_2'], dtype='object')

In [230]:
del(result['index'])

In [231]:
result.tail()

,source_1,source_2
2316,12129,30164
2317,12130,37661
2318,12141,39826
2319,12183,36565
2320,12188,7013


In [232]:
result.to_csv('runoharris215@gmail.com.csv')